In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [56]:
import torch
import cv2
import numpy as np

def load_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

def detect_vehicles(model, image, confidence_threshold=0.5):
    results = model(image)
    results = results.pandas().xyxy[0]  # Results as DataFrame
    vehicles = results[(results['name'].isin(['car', 'truck'])) & (results['confidence'] >= confidence_threshold)]
    bboxes = []
    for _, vehicle in vehicles.iterrows():
        bbox = vehicle[['xmin', 'ymin', 'xmax', 'ymax']].astype(int).values.tolist()
        bboxes.append(bbox)
    return bboxes

def draw_dimension(image, bbox, dimension, font_scale=1, font_thickness=2):
    x1, y1, x2, y2 = bbox
    cv2.putText(image, f"{dimension:.2f}m", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

def draw_distance(image, bbox1, bbox2, distance, font_scale=1, font_thickness=2):
    center_x1 = (bbox1[0] + bbox1[2]) // 2
    center_y1 = (bbox1[1] + bbox1[3]) // 2
    center_x2 = (bbox2[0] + bbox2[2]) // 2
    center_y2 = (bbox2[1] + bbox2[3]) // 2
    center_x = (center_x1 + center_x2) // 2
    center_y = (center_y1 + center_y2) // 2
    cv2.putText(image, f"{distance:.2f}m", (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 0, 0), font_thickness)

def save_annotated_image(image, bboxes, distances, output_path):
    image_annotated = image.copy()
    for bbox, dist in zip(bboxes, distances):
        x1, y1, x2, y2 = bbox
        cv2.rectangle(image_annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
    for i in range(len(bboxes) - 1):
        bbox1 = bboxes[i]
        bbox2 = bboxes[i + 1]
        distance = distances[i]
        draw_distance(image_annotated, bbox1, bbox2, distance)
    cv2.imwrite(output_path, image_annotated)

def compute_distances(bboxes, camera_params):
    distances = []
    pixel_width_ratio = camera_params[-1]  # Extract pixel width ratio from camera_params
    for i in range(len(bboxes) - 1):
        bbox1 = bboxes[i]
        bbox2 = bboxes[i + 1]
        distance = compute_distance(bbox1, bbox2, camera_params, pixel_width_ratio)
        distances.append(distance*100)  # Multiply by 100 to show distances in centimeters
    return distances

def compute_distance(bbox1, bbox2, camera_params, pixel_width_ratio):
    # Unpack camera parameters
    f, cx, cy, D = camera_params[:-1]  # Exclude pixel_width_ratio

    # Compute pixel distance between bounding boxes
    center_x1 = (bbox1[0] + bbox1[2]) // 2
    center_x2 = (bbox2[0] + bbox2[2]) // 2
    pixel_distance = abs(center_x2 - center_x1)

    # Convert pixel distance to dimensions using pixel_width_ratio
    distance = (f * pixel_distance) / (2 * cx * D) * pixel_width_ratio

    return distance

# Example usage
model = load_model()
image_path = '/content/front_parking.jpg'
output_annotated = 'distance_annotated.jpg'

image = cv2.imread(image_path)
bboxes = detect_vehicles(model, image)

# Example camera parameters (focal length, optical center x, optical center y, distance to vehicle, pixel width ratio)
camera_params = (1000, 960, 540, 10000, 1.0)  # Example values, assuming 1 pixel width = 1 meter

if bboxes:
    print(f"Detected {len(bboxes)} vehicles.")
    distances = compute_distances(bboxes, camera_params)
    print("Distances between consecutive vehicles:", distances)

    # Save annotated image with distances as vehicle numbers
    save_annotated_image(image, bboxes, distances, output_annotated)
    print(f"Annotated image saved at {output_annotated}")
else:
    print("No vehicles detected in the image.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-14 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected 4 vehicles.
Distances between consecutive vehicles: [2.7916666666666665, 4.151041666666667, 2.442708333333333]
Annotated image saved at distance_annotated.jpg


In [57]:
import torch
import cv2
import numpy as np

def load_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

def detect_vehicles(model, image, confidence_threshold=0.5):
    results = model(image)
    results = results.pandas().xyxy[0]  # Results as DataFrame
    vehicles = results[(results['name'].isin(['car', 'truck'])) & (results['confidence'] >= confidence_threshold)]
    bboxes = []
    for _, vehicle in vehicles.iterrows():
        bbox = vehicle[['xmin', 'ymin', 'xmax', 'ymax']].astype(int).values.tolist()
        bboxes.append(bbox)
    return bboxes

def draw_dimension(image, bbox, dimension, font_scale=1, font_thickness=2):
    x1, y1, x2, y2 = bbox
    cv2.putText(image, f"{dimension:.2f}m", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

def draw_distance(image, bbox1, bbox2, distance, font_scale=1, font_thickness=2):
    center_x1 = (bbox1[0] + bbox1[2]) // 2
    center_y1 = (bbox1[1] + bbox1[3]) // 2
    center_x2 = (bbox2[0] + bbox2[2]) // 2
    center_y2 = (bbox2[1] + bbox2[3]) // 2
    center_x = (center_x1 + center_x2) // 2
    center_y = (center_y1 + center_y2) // 2
    cv2.putText(image, f"{distance:.2f}m", (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 0, 0), font_thickness)

def save_annotated_image(image, bboxes, distances, output_path):
    image_annotated = image.copy()
    for bbox, dist in zip(bboxes, distances):
        x1, y1, x2, y2 = bbox
        cv2.rectangle(image_annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
    for i in range(len(bboxes) - 1):
        bbox1 = bboxes[i]
        bbox2 = bboxes[i + 1]
        distance = distances[i]
        draw_distance(image_annotated, bbox1, bbox2, distance)
    cv2.imwrite(output_path, image_annotated)

def compute_distances(bboxes, camera_params):
    distances = []
    pixel_width_ratio = camera_params[-1]  # Extract pixel width ratio from camera_params
    for i in range(len(bboxes) - 1):
        bbox1 = bboxes[i]
        bbox2 = bboxes[i + 1]
        distance = compute_distance(bbox1, bbox2, camera_params, pixel_width_ratio)
        distances.append(distance*100)  # Multiply by 100 to show distances in centimeters
    return distances

def compute_distance(bbox1, bbox2, camera_params, pixel_width_ratio):
    # Unpack camera parameters
    f, cx, cy, D = camera_params[:-1]  # Exclude pixel_width_ratio

    # Compute pixel distance between bounding boxes
    center_x1 = (bbox1[0] + bbox1[2]) // 2
    center_x2 = (bbox2[0] + bbox2[2]) // 2
    pixel_distance = abs(center_x2 - center_x1)

    # Convert pixel distance to dimensions using pixel_width_ratio
    distance = (f * pixel_distance) / (2 * cx * D) * pixel_width_ratio

    return distance

# Example usage
model = load_model()
image_path = '/content/front_parking1.jpg'
output_annotated = 'distance_annotated.jpg'

image = cv2.imread(image_path)
bboxes = detect_vehicles(model, image)

# Example camera parameters (focal length, optical center x, optical center y, distance to vehicle, pixel width ratio)
camera_params = (1000, 960, 540, 10000, 1.0)  # Example values, assuming 1 pixel width = 1 meter

if bboxes:
    print(f"Detected {len(bboxes)} vehicles.")
    distances = compute_distances(bboxes, camera_params)
    print("Distances between consecutive vehicles:", distances)

    # Save annotated image with distances as vehicle numbers
    save_annotated_image(image, bboxes, distances, output_annotated)
    print(f"Annotated image saved at {output_annotated}")
else:
    print("No vehicles detected in the image.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-14 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected 3 vehicles.
Distances between consecutive vehicles: [2.7916666666666665, 4.151041666666667]
Annotated image saved at distance_annotated.jpg
